# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6b6055fe20>
├── 'a' --> tensor([[ 0.6729,  0.5967, -1.1591],
│                   [-0.3069,  0.9622, -0.9610]])
└── 'x' --> <FastTreeValue 0x7f6b605cc100>
    └── 'c' --> tensor([[ 0.7303,  0.5949,  0.1081,  0.0708],
                        [-1.1417,  0.7616, -0.5957, -1.3031],
                        [ 0.0089, -1.4327,  0.5378, -0.4435]])

In [4]:
t.a

tensor([[ 0.6729,  0.5967, -1.1591],
        [-0.3069,  0.9622, -0.9610]])

In [5]:
%timeit t.a

62.7 ns ± 0.0517 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6b6055fe20>
├── 'a' --> tensor([[ 1.1962, -1.2772, -0.6416],
│                   [-0.4416, -0.5475, -1.3026]])
└── 'x' --> <FastTreeValue 0x7f6b605cc100>
    └── 'c' --> tensor([[ 0.7303,  0.5949,  0.1081,  0.0708],
                        [-1.1417,  0.7616, -0.5957, -1.3031],
                        [ 0.0089, -1.4327,  0.5378, -0.4435]])

In [7]:
%timeit t.a = new_value

73.5 ns ± 0.0333 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6729,  0.5967, -1.1591],
               [-0.3069,  0.9622, -0.9610]]),
    x: Batch(
           c: tensor([[ 0.7303,  0.5949,  0.1081,  0.0708],
                      [-1.1417,  0.7616, -0.5957, -1.3031],
                      [ 0.0089, -1.4327,  0.5378, -0.4435]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6729,  0.5967, -1.1591],
        [-0.3069,  0.9622, -0.9610]])

In [11]:
%timeit b.a

58.1 ns ± 0.103 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6888, -0.1187, -0.3748],
               [ 0.3583,  0.2816,  0.8495]]),
    x: Batch(
           c: tensor([[ 0.7303,  0.5949,  0.1081,  0.0708],
                      [-1.1417,  0.7616, -0.5957, -1.3031],
                      [ 0.0089, -1.4327,  0.5378, -0.4435]]),
       ),
)

In [13]:
%timeit b.a = new_value

504 ns ± 0.124 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

807 ns ± 0.776 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 19.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 299 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 492 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6abce27a60>
├── 'a' --> tensor([[[ 0.6729,  0.5967, -1.1591],
│                    [-0.3069,  0.9622, -0.9610]],
│           
│                   [[ 0.6729,  0.5967, -1.1591],
│                    [-0.3069,  0.9622, -0.9610]],
│           
│                   [[ 0.6729,  0.5967, -1.1591],
│                    [-0.3069,  0.9622, -0.9610]],
│           
│                   [[ 0.6729,  0.5967, -1.1591],
│                    [-0.3069,  0.9622, -0.9610]],
│           
│                   [[ 0.6729,  0.5967, -1.1591],
│                    [-0.3069,  0.9622, -0.9610]],
│           
│                   [[ 0.6729,  0.5967, -1.1591],
│                    [-0.3069,  0.9622, -0.9610]],
│           
│                   [[ 0.6729,  0.5967, -1.1591],
│                    [-0.3069,  0.9622, -0.9610]],
│           
│                   [[ 0.6729,  0.5967, -1.1591],
│                    [-0.3069,  0.9622, -0.9610]]])
└── 'x' --> <FastTreeValue 0x7f6abd094280>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6aab7f1c10>
├── 'a' --> tensor([[ 0.6729,  0.5967, -1.1591],
│                   [-0.3069,  0.9622, -0.9610],
│                   [ 0.6729,  0.5967, -1.1591],
│                   [-0.3069,  0.9622, -0.9610],
│                   [ 0.6729,  0.5967, -1.1591],
│                   [-0.3069,  0.9622, -0.9610],
│                   [ 0.6729,  0.5967, -1.1591],
│                   [-0.3069,  0.9622, -0.9610],
│                   [ 0.6729,  0.5967, -1.1591],
│                   [-0.3069,  0.9622, -0.9610],
│                   [ 0.6729,  0.5967, -1.1591],
│                   [-0.3069,  0.9622, -0.9610],
│                   [ 0.6729,  0.5967, -1.1591],
│                   [-0.3069,  0.9622, -0.9610],
│                   [ 0.6729,  0.5967, -1.1591],
│                   [-0.3069,  0.9622, -0.9610]])
└── 'x' --> <FastTreeValue 0x7f6aab811af0>
    └── 'c' --> tensor([[ 0.7303,  0.5949,  0.1081,  0.0708],
                        [-1.1417,  0.7616, -0.5957, -1.3031],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 88.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.6 µs ± 45.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.7303,  0.5949,  0.1081,  0.0708],
                       [-1.1417,  0.7616, -0.5957, -1.3031],
                       [ 0.0089, -1.4327,  0.5378, -0.4435]],
              
                      [[ 0.7303,  0.5949,  0.1081,  0.0708],
                       [-1.1417,  0.7616, -0.5957, -1.3031],
                       [ 0.0089, -1.4327,  0.5378, -0.4435]],
              
                      [[ 0.7303,  0.5949,  0.1081,  0.0708],
                       [-1.1417,  0.7616, -0.5957, -1.3031],
                       [ 0.0089, -1.4327,  0.5378, -0.4435]],
              
                      [[ 0.7303,  0.5949,  0.1081,  0.0708],
                       [-1.1417,  0.7616, -0.5957, -1.3031],
                       [ 0.0089, -1.4327,  0.5378, -0.4435]],
              
                      [[ 0.7303,  0.5949,  0.1081,  0.0708],
                       [-1.1417,  0.7616, -0.5957, -1.3031],
                       [ 0.0089, -1.4327,  0.5378, -0.4435]],

In [26]:
%timeit Batch.stack(batches)

76.9 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.7303,  0.5949,  0.1081,  0.0708],
                      [-1.1417,  0.7616, -0.5957, -1.3031],
                      [ 0.0089, -1.4327,  0.5378, -0.4435],
                      [ 0.7303,  0.5949,  0.1081,  0.0708],
                      [-1.1417,  0.7616, -0.5957, -1.3031],
                      [ 0.0089, -1.4327,  0.5378, -0.4435],
                      [ 0.7303,  0.5949,  0.1081,  0.0708],
                      [-1.1417,  0.7616, -0.5957, -1.3031],
                      [ 0.0089, -1.4327,  0.5378, -0.4435],
                      [ 0.7303,  0.5949,  0.1081,  0.0708],
                      [-1.1417,  0.7616, -0.5957, -1.3031],
                      [ 0.0089, -1.4327,  0.5378, -0.4435],
                      [ 0.7303,  0.5949,  0.1081,  0.0708],
                      [-1.1417,  0.7616, -0.5957, -1.3031],
                      [ 0.0089, -1.4327,  0.5378, -0.4435],
                      [ 0.7303,  0.5949,  0.1081,  0.0708],
                   

In [28]:
%timeit Batch.cat(batches)

164 µs ± 175 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

319 µs ± 3.68 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
